# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [44]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [45]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("data/ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [46]:
# Your code here
scaler = StandardScaler()
# Scale X_train and X_val using StandardScaler
X_train_scaled =  pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
X_val_Scaled = pd.DataFrame(scaler.transform(X_val),columns=X_train.columns)
# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
# X_train_scaled.columns=X_train.columns
X_train_scaled


,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820
...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820


In [47]:
# Your code here
regression = LinearRegression()
# Create a LinearRegression model and fit it on scaled training data
regression.fit(X_train_scaled,y_train)

# Calculate a baseline r-squared score on training data
regression.score(X_train_scaled,y_train)


0.7868344817421309

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [48]:

# Set up data structure
# (Here we are using a list of tuples, but you could use a dictionary,
# a list of lists, some other structure. Whatever makes sense to you.)
interactions = []

# Find combinations of columns and loop over them
column_pairs = list(combinations(X_train.columns, 2))
for (col1, col2) in column_pairs:
    # Make copies of X_train and X_val
    features_train = X_train.copy()
    features_val = X_val.copy()
    
    # Add interaction term to data
    features_train["interaction"] = features_train[col1] * features_train[col2]
    features_val["interaction"] = features_val[col1] * features_val[col2]
    
    # Find r-squared score (fit on training data, evaluate on test data)
    score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)
    
    # Append to data structure
    interactions.append(((col1, col2), score))

# Sort and subset the data structure to find the top 7
top_7_interactions = sorted(interactions, key=lambda record: record[1], reverse=True)[:7]
print("Top 7 interactions:")
print(top_7_interactions)

Top 7 interactions:
[(('LotArea', '1stFlrSF'), 0.7211105669453352), (('LotArea', 'TotalBsmtSF'), 0.7071649211190745), (('LotArea', 'GrLivArea'), 0.6690980828075868), (('LotArea', 'Fireplaces'), 0.6529699515655466), (('2ndFlrSF', 'TotRmsAbvGrd'), 0.6472994890405049), (('OverallCond', 'TotalBsmtSF'), 0.6429019879233231), (('OverallQual', '2ndFlrSF'), 0.6422324294284469)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [49]:

# Loop over top 7 interactions
for record in top_7_interactions:
    # Extract column names from data structure
    col1, col2 = record[0]
    
    # Construct new column name
    new_col_name = col1 + "_" + col2
    
    # Add new column to X_train and X_val
    X_train[new_col_name] = X_train[col1] * X_train[col2]
    X_val[new_col_name] = X_val[col1] * X_val[col2]
    
X_train

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
518,9531,6,5,794,882,914,1796,7,546,0,8406342,7567614,17117676,0,6398,3970,5484
236,8773,7,5,1414,1414,0,1414,6,494,0,12405022,12405022,12405022,0,0,7070,0
38,7922,5,7,1057,1057,0,1057,5,246,0,8373554,8373554,8373554,0,0,7399,0
1034,6305,5,7,920,954,0,954,5,240,1,6014970,5800600,6014970,6305,0,6440,0
520,10800,4,7,0,694,600,1294,7,0,0,7495200,0,13975200,0,4200,0,2400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,4426,6,5,848,848,0,848,3,420,1,3753248,3753248,3753248,4426,0,4240,0
1309,7153,6,5,1278,1294,0,1294,6,496,0,9255982,9141534,9255982,0,0,6390,0
1237,12393,7,5,847,847,1101,1948,8,434,1,10496871,10496871,24141564,12393,8808,4235,7707
214,10900,6,7,689,689,703,1392,6,299,0,7510100,7510100,15172800,0,4218,4823,4218


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [53]:

# Set up data structure
polynomials = []

# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in (2, 3, 4):
        
        # Make a copy of X_train and X_val
        features_train = X_train.copy()
        features_val = X_val.copy()
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        col_transformed_train = pd.DataFrame(poly.fit_transform(features_train[[col]]))
        col_transformed_val = pd.DataFrame(poly.transform(features_val[[col]]))
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        features_train = pd.concat([features_train.drop(col, axis=1), col_transformed_train], axis=1)
        features_val = pd.concat([features_val.drop(col, axis=1), col_transformed_val], axis=1)
    
        # Find r-squared score
        score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)
    
        # Append to data structure
        polynomials.append((col, degree, score))
    
# Sort and subset the data structure to find the top 7
top_7_polynomials = sorted(polynomials, key=lambda record: record[-1], reverse=True)[:7]
print("Top 7 polynomials:")
print(top_7_polynomials)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [54]:

# Set up data structure
polynomials = []

# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in (2, 3, 4):
        
        # Make a copy of X_train and X_val
        features_train = X_train.copy()
        features_val = X_val.copy()
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        col_transformed_train = pd.DataFrame(poly.fit_transform(features_train[[col]]))
        col_transformed_val = pd.DataFrame(poly.transform(features_val[[col]]))
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        features_train = pd.concat([features_train.drop(col, axis=1), col_transformed_train], axis=1)
        features_val = pd.concat([features_val.drop(col, axis=1), col_transformed_val], axis=1)
    
        # Find r-squared score
        score = LinearRegression().fit(features_train, y_train).score(features_val, y_val)
    
        # Append to data structure
        polynomials.append((col, degree, score))
    
# Sort and subset the data structure to find the top 7
top_7_polynomials = sorted(polynomials, key=lambda record: record[-1], reverse=True)[:7]
print("Top 7 polynomials:")
print(top_7_polynomials)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [57]:
# Your code here
X_train.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
518,9531,6,5,794,882,914,1796,7,546,0,8406342,7567614,17117676,0,6398,3970,5484
236,8773,7,5,1414,1414,0,1414,6,494,0,12405022,12405022,12405022,0,0,7070,0
38,7922,5,7,1057,1057,0,1057,5,246,0,8373554,8373554,8373554,0,0,7399,0
1034,6305,5,7,920,954,0,954,5,240,1,6014970,5800600,6014970,6305,0,6440,0
520,10800,4,7,0,694,600,1294,7,0,0,7495200,0,13975200,0,4200,0,2400


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [55]:
lr = LinearRegression()
lr.fit(X_train, y_train)

print("Train R^2:", lr.score(X_train, y_train))
print("Validation R^2: ", lr.score(X_val, y_val))

Train R^2: 0.8003632679622383
Validation R^2:  0.746820544087928


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [58]:
# Your code here
for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)

    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    print("Train R^2:", lr.score(X_rfe_train, y_train))
    print("Test R^2: ", lr.score(X_rfe_val, y_val))
    print(f"Using {n} out of {X_train.shape[1]} features")
    print()


Train R^2: 0.728068322986812
Test R^2:  0.6817204948664276
Using 5 out of 17 features

Train R^2: 0.7896103579371743
Test R^2:  0.644397675375793
Using 10 out of 17 features

Train R^2: 0.7989937484918626
Test R^2:  0.7376607606613379
Using 15 out of 17 features

Train R^2: 0.8003632679622383
Test R^2:  0.746820544087928
Using 20 out of 17 features

Train R^2: 0.8003632679622383
Test R^2:  0.746820544087928
Using 25 out of 17 features



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [59]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    print("Train R^2:", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Using { 26 - (sum(abs(lasso.coef_) < 10**(-10)))} out of {X_train.shape[1]} features")
    print("and an alpha of", alpha)
    print()

Train R^2: 0.800363264314369
Validation R^2:  0.7468229842861163
Using 26 out of 17 features
and an alpha of 1

Train R^2: 0.8003629334472134
Validation R^2:  0.7468506014145233
Using 26 out of 17 features
and an alpha of 10

Train R^2: 0.8003299146607135
Validation R^2:  0.7471002280260219
Using 26 out of 17 features
and an alpha of 100

Train R^2: 0.7992360773736773
Validation R^2:  0.744363647682514
Using 24 out of 17 features
and an alpha of 1000

Train R^2: 0.7737586226785165
Validation R^2:  0.6945153994739204
Using 22 out of 17 features
and an alpha of 10000



Compare the results. Which features would you choose to use?

In [60]:
# Your written answer here
"""
For RFE the model with the best validation R-Squared was using 20 features

For Lasso the model with the best validation R-Squared was using an alpha of 10000

The Lasso result was a bit better so let's go with that and the 12 features
that it selected
"""

"\nFor RFE the model with the best validation R-Squared was using 20 features\n\nFor Lasso the model with the best validation R-Squared was using an alpha of 10000\n\nThe Lasso result was a bit better so let's go with that and the 12 features\nthat it selected\n"

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [61]:
# Your code here
# Scale X_test
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

# Add interactions to X_test
for record in top_7_interactions:
    col1, col2 = record[0]
    new_col_name = col1 + "_" + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]

# Add polynomials to X_val
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)



NameError: name 'top_polynomials' is not defined

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [62]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))



R-Squared: -9.024393130607201
MSE: 70223971942.05072


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.